<a href="https://colab.research.google.com/github/ohhhanna/twitter-sentiment-analysis/blob/main/twt_snt_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pickle
import re
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.cuda.empty_cache()

In [3]:
x_train_file = open('/content/drive/My Drive/X_train.pkl', 'rb')
X_train = pickle.load(x_train_file)

x_test_file = open('/content/drive/My Drive/X_test.pkl', 'rb')
X_test = pickle.load(x_test_file)

y_train_file = open('/content/drive/My Drive/Y_train.pkl', 'rb')
Y_train = pickle.load(y_train_file)

y_test_file = open('/content/drive/My Drive/Y_test.pkl', 'rb')
Y_test = pickle.load(y_test_file)

x_val_file = open('/content/drive/My Drive/X_val.pkl', 'rb')
X_val = pickle.load(x_val_file)

y_val_file = open('/content/drive/My Drive/Y_val.pkl', 'rb')
Y_val = pickle.load(y_val_file)

#rb= read binary

In [4]:
words = []
idx = 0
word2idx = {}
vectors = []

with open(f'/content/drive/MyDrive/Colab Notebooks/glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        #we are just spliting the words and aooending them, giving them an index and embedding them with float64 and outting them in a vector

        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float64)

        vectors.append(vect)
#putting the vectors in glove
glove = {w: vectors[word2idx[w]] for w in words}



In [5]:
# Save GloVe dictionary.
glove_file = open("/content/drive/My Drive/glove_dict_50d.pkl", "wb")
pickle.dump(glove,glove_file)
glove_file.close()

# Load GloVe dictionary.
glove_file = open("/content/drive/My Drive/glove_dict_50d.pkl", "rb")
glove = pickle.load(glove_file)
# glove_file.close()

In [6]:
def Vectors_from_Glove(dataset):
  matrix_len = dataset['text'].str.split().str.len().sum()
  # print(matrix_len)
  words_found = 0
  words = 0
  mean_tweets = []

  for i,row in enumerate(dataset['text']):
    splited_words = str(row).split(' ')
    weight_matrix = np.zeros((len(splited_words),50))
    for j,word in enumerate(splited_words):
      words += 1
      if word in glove:
        weight_matrix[j] = glove[word]
        words_found += 1
    mean_tweets.append(np.mean(weight_matrix,axis=0))

  return np.array(mean_tweets), words_found, matrix_len

In [7]:
gloveX_train, words_found, matrix_len = Vectors_from_Glove(pd.DataFrame(X_train))
print(gloveX_train.shape)
# print(type(gloveX_train))
# print(len(gloveX_train[0]))
# print(gloveX_train[0])
print("Ratio of words from x_train that have founded in Glove: ", round(words_found/matrix_len,2))

gloveX_val, words_found, matrix_len = Vectors_from_Glove(pd.DataFrame(X_val))
print("Ratio of words from x_val that have founded in Glove: ", round(words_found/matrix_len,2))

gloveX_test, words_found, matrix_len = Vectors_from_Glove(pd.DataFrame(X_test))
print("Ratio of words from x_test that have founded in Glove: ", round(words_found/matrix_len,2))

(495012, 50)
Ratio of words from x_train that have founded in Glove:  0.93
Ratio of words from x_val that have founded in Glove:  0.92
Ratio of words from x_test that have founded in Glove:  0.93


In [8]:
# Convert X datasets to tensors with function convert_to_tensor.
gloveX_train = torch.tensor(gloveX_train)
gloveX_val = torch.tensor(gloveX_val)
gloveX_test = torch.tensor(gloveX_test)

In [9]:
# Convert Y labels to tensors with torch.squeeze.
Y_train = torch.squeeze(torch.from_numpy(Y_train.to_numpy()).float())
Y_val = torch.squeeze(torch.from_numpy(Y_val.to_numpy()).float())
Y_test = torch.squeeze(torch.from_numpy(Y_test.to_numpy()).float())

In [10]:
#check cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

class LSTM_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTM_Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x.view(len(x), 1, -1))
        output = self.fc(lstm_out.view(len(x), -1))
        output = self.sigmoid(output)
        return output

# Assuming gloveX_train is your input data
input_dim = gloveX_train.shape[1]
hidden_dim = 64
output_dim = 1

# Instantiate model, loss function, and optimizer
lstm_model = LSTM_Model(input_dim, hidden_dim, output_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=1e-4, weight_decay=1e-4)

# Move model to GPU if available
lstm_model.to(device)
criterion = criterion.to(device)


In [20]:
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have your training data in gloveX_train and corresponding labels in Y_train
train_data = TensorDataset(gloveX_train, Y_train)

# Define batch size
batch_size = 64

# Create DataLoader
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Define number of epochs
num_epochs = 80

# Training loop
for epoch in range(num_epochs):
    lstm_model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Convert inputs to torch.float32
        inputs = inputs.to(torch.float32)

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = lstm_model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))  # Ensure labels have correct shape
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)

        # Calculate accuracy
        predicted_labels = (outputs > 0.5).float()
        correct_predictions += (predicted_labels == labels.unsqueeze(1)).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")


Epoch 1/80, Loss: 0.6234, Accuracy: 0.6544
Epoch 2/80, Loss: 0.6039, Accuracy: 0.6703
Epoch 3/80, Loss: 0.6027, Accuracy: 0.6708
Epoch 4/80, Loss: 0.6019, Accuracy: 0.6716
Epoch 5/80, Loss: 0.6012, Accuracy: 0.6718
Epoch 6/80, Loss: 0.6005, Accuracy: 0.6724
Epoch 7/80, Loss: 0.5998, Accuracy: 0.6731
Epoch 8/80, Loss: 0.5991, Accuracy: 0.6734
Epoch 9/80, Loss: 0.5984, Accuracy: 0.6745
Epoch 10/80, Loss: 0.5975, Accuracy: 0.6750
Epoch 11/80, Loss: 0.5967, Accuracy: 0.6758
Epoch 12/80, Loss: 0.5959, Accuracy: 0.6767
Epoch 13/80, Loss: 0.5952, Accuracy: 0.6774
Epoch 14/80, Loss: 0.5944, Accuracy: 0.6778
Epoch 15/80, Loss: 0.5937, Accuracy: 0.6785
Epoch 16/80, Loss: 0.5930, Accuracy: 0.6797
Epoch 17/80, Loss: 0.5922, Accuracy: 0.6797
Epoch 18/80, Loss: 0.5916, Accuracy: 0.6805
Epoch 19/80, Loss: 0.5909, Accuracy: 0.6812
Epoch 20/80, Loss: 0.5903, Accuracy: 0.6817
Epoch 21/80, Loss: 0.5898, Accuracy: 0.6821
Epoch 22/80, Loss: 0.5893, Accuracy: 0.6822
Epoch 23/80, Loss: 0.5887, Accuracy: 0.68

In [23]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have your test data in gloveX_test and corresponding labels in Y_test
test_data = TensorDataset(gloveX_test, Y_test)

# Define batch size
batch_size = 64

# Create DataLoader
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Testing loop
lstm_model.eval()
test_loss = 0.0
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(torch.float32)
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = lstm_model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))  # Ensure labels have correct shape

        test_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)

        # Calculate accuracy
        predicted_labels = (outputs > 0.5).float()
        correct_predictions += (predicted_labels == labels.unsqueeze(1)).sum().item()

test_loss = test_loss / total_samples
test_accuracy = correct_predictions / total_samples

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Test Loss: 0.5676, Test Accuracy: 0.6960
